In [ ]:
import pandas as pd 

In [1]:
import pandas as pd

df = pd.read_csv('../data/movie_subset_with_descriptors.csv')

df.head()

df.columns

df.info()

df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      109 non-null    int64 
 1   Title             109 non-null    object
 2   Origin/Ethnicity  109 non-null    object
 3   Director          109 non-null    object
 4   Cast              109 non-null    object
 5   Genre             109 non-null    object
 6   Wiki Page         109 non-null    object
 7   Plot              109 non-null    object
 8   descriptors       109 non-null    object
dtypes: int64(1), object(8)
memory usage: 7.8+ KB


,Release Year
count,109.000000
mean,2013.000000
std,2.231923
min,2010.000000
25%,2011.000000
50%,2013.000000
75%,2015.000000
max,2017.000000


In [2]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,descriptors
0,2015,Black Mass,American,Scott Cooper,Johnny Depp\r\nBenedict Cumberbatch\r\nJoel Ed...,action,https://en.wikipedia.org/wiki/Black_Mass_(film),"In 1975, James ""Whitey"" Bulger, leader of the ...","Here is a list of 50 unique, concise tag-style..."
1,2012,Air Collision,American,Liz Adams,"Reginald VelJohnson, Jordan Ladd, Gerald Webb",action,https://en.wikipedia.org/wiki/Air_Collision,After a major electromagnetic storm cripples c...,"Here is a list of 50 unique, concise tag-style..."
2,2015,Taken 3,American,Olivier Megaton,Liam Neeson\r\nForest Whitaker\r\nFamke Jansse...,action,https://en.wikipedia.org/wiki/Taken_3,Former covert operative Bryan Mills (Liam Nees...,"Here is a list of 50 unique, concise tag-style..."
3,2015,The Divergent Series: Insurgent,American,Robert Schwentke,Shailene Woodley\r\nTheo James\r\nOctavia Spen...,action,https://en.wikipedia.org/wiki/The_Divergent_Se...,Five days after the assault on the Abnegation ...,"Here is a comma-separated list of 50 unique, c..."
4,2013,White House Down,American,Roland Emmerich,"Channing Tatum, Jamie Foxx, Maggie Gyllenhaal,...",action comedy,https://en.wikipedia.org/wiki/White_House_Down,President of the United States James Sawyer (J...,"Here is a list of 50 unique, concise tag-style..."


In [5]:
# Just keep the df columns Plot, Descriptiors, and Title

df = df[['Plot', 'descriptors', 'Title']]


In [6]:
df.head()

,Plot,descriptors,Title
0,"In 1975, James ""Whitey"" Bulger, leader of the ...","Here is a list of 50 unique, concise tag-style...",Black Mass
1,After a major electromagnetic storm cripples c...,"Here is a list of 50 unique, concise tag-style...",Air Collision
2,Former covert operative Bryan Mills (Liam Nees...,"Here is a list of 50 unique, concise tag-style...",Taken 3
3,Five days after the assault on the Abnegation ...,"Here is a comma-separated list of 50 unique, c...",The Divergent Series: Insurgent
4,President of the United States James Sawyer (J...,"Here is a list of 50 unique, concise tag-style...",White House Down


In [8]:
import re
import pandas as pd


# 2. Define a cleaning function
def clean_descriptors(raw: str) -> str:
    if pd.isna(raw):
        return ""
    # Remove anything up through the first colon/newline
    cleaned = re.sub(r'^[^A-Za-z0-9]*.*?:\s*', "", raw, count=1)
    # Normalize newlines to commas, split & strip
    parts = [p.strip() for p in cleaned.replace("\n", ", ").split(",")]
    # Re-join into a clean comma-separated string
    return ", ".join([p for p in parts if p])

# 3. Apply it
df["descriptors"] = df["descriptors"].apply(clean_descriptors)

# 4. Save out the cleaned CSV
df.to_csv("movie_subset_descriptors_cleaned.csv", index=False)

print("✅ Cleaned descriptors saved to movie_subset_descriptors_cleaned.csv")

✅ Cleaned descriptors saved to movie_subset_descriptors_cleaned.csv


C:\Users\Sanket\AppData\Local\Temp\ipykernel_66244\3843150432.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["descriptors"] = df["descriptors"].apply(clean_descriptors)


In [ ]:
# Now create store it in a pinecone index
from pinecone import Pinecone
import time
import json
import random
import itertools
import os
import yaml


with open("secrets.yaml", 'r') as stream:
    keys = yaml.safe_load(stream)
    

def chunks(iterable, batch_size=200):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))


def create_vector_database( batch_size: int = 95, index_name: str = "movies", namespace: str = "example-namespace"):
# Initialize a Pinecone client with your API key
    pc = Pinecone(api_key='pcsk_3R7ydo_J5XoRYLTZpVreUhC6UuUbjPfB258sBdqFX8VKNi9LnCJCgeugyPbwezXB6my4wP')
   

    data = pd.read_json('movies.json')

    # Create a dense index with integrated embedding (or connect if exists)
    if not pc.has_index(index_name):
        print(f"Creating index '{index_name}' with integrated embeddings for 'llama-text-embed-v2'...")
        pc.create_index_for_model(
            name=index_name,
            cloud="aws",
            region="us-east-1",
            embed={
                "model":"llama-text-embed-v2",
                "field_map":{"text": "chunk_text"} # Pinecone will look for 'chunk_text' in your records
            }
        )
        print(f"Index '{index_name}' created. It might take a moment to be ready.")
        time.sleep(10) # Give a moment for the index to initialize
    else:
        print(f"Index '{index_name}' already exists. Connecting to it.")

    dense_index = pc.Index(index_name)

    # Upsert the records into a namespace
    print(f"Upserting {len(records_to_upsert)} records into namespace {index_name} in batches...")

    # Define batch size, matching your chunks function or Pinecone limits (e.g., 100-200 for upsert_records)
    total_upserted_count = 0

    for record_chunk in chunks(records_to_upsert, batch_size=batch_size):
        try:
            print(f"Upserting batch of {len(record_chunk)} records...")
            dense_index.upsert_records(namespace="example-namespace", records=list(record_chunk))
            total_upserted_count += len(record_chunk)
            print(f"Successfully upserted batch. Total upserted so far: {total_upserted_count}")
        except Exception as e:
            print(f"Error upserting batch: {e}")
            # Optionally, add more sophisticated error handling here, like retries or logging failed batches

    print(f"Successfully upserted {total_upserted_count} records in total to namespace 'example-namespace'.")


